<h1> Activate chs_env virtual enviroment </h1>
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy Unrestricted <br>
chess_env\Scripts\activate

In [3]:
# Importing all relevant and necessary modules
import numpy as np 
import pandas as pd 
import os
import chess
import math
import matplotlib.pyplot as plt
import ollama


In [4]:
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def gen_prompt2(board,moves, n=10):
    return f"You are playing chess and the board state is {board}. Here you are given a list of legal moves {moves}. You have to chose the best move to make next from the list of legal moves and provide the move exactly as they are given to you. What is your next move given in UCI notation?"
    
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def prompt_move(prompt,board):
    response = ollama.chat(model='llama2:13b', messages=[
            {'role': 'user','content': prompt,},])
    move = response['message']['content']
    for k in range(len(move)):
        if move[k:k+4] in gen_legal_moves(board):
            move = move[k:k+4]
            break
    return move

In [5]:
df = pd.read_csv(r"chessData.csv")

In [6]:
def prompt_ensemble(size,fen):
    moves = {}  #Dictionary to store the moves and their frequencies
    for i in range(size):
        board = chess.Board(fen)
        move = prompt_move(gen_prompt2(board,gen_legal_moves(board)),board)
        if move in moves:
            moves[move] += 1 #Increment the frequency of the move
        else:
            moves[move] = 1 #Add the move to the dictionary
    return moves

def ensemble_manager(df,sizes):
    ensemble = {}
    for size in sizes:
        ensemble[size] = prompt_ensemble(size,df['FEN'][0]) #df['FEN'][0] is the first FEN in the dataset change for iteration over all FENs

    return ensemble

In [7]:
fen = df['FEN'][0]
size = 10
ensemble = prompt_ensemble(size,fen)
ensemble

In [ ]:
gen_legal_moves(chess.Board(fen))